In [ ]:
import os
import re
import uuid
import fitz
import json
import numpy as np
from pathlib import Path
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from sentence_transformers import util


# ==========================
# CONFIGURATION
# ==========================

PROJECT_ROOT = Path(r"/home/jagannath/myPocketLawyer")

DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = PROJECT_ROOT / "processed_data"
VECTORSTORE_DIR = PROJECT_ROOT / "chroma_db"

EMBEDDING_MODEL = "BAAI/bge-large-en-v1.5"

/home/jagannath/myPocketLawyer/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

def load_pdf_text(pdf_path: Path) -> str:
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text")
    return text

def clean_legal_text(text: str) -> str:
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'–', '-', text)
    return text.strip()

def segment_legal_sections(text: str) -> list:
    """
    Split legal PDFs into chapters/parts and sections.
    Handles:
      - Chapter/Part with numbers or Roman numerals
      - Multi-line chapter titles
      - Sections kept together
      - Optional clauses preserved but not split line-by-line
    """
    sections = []
    current_chapter = None
    current_section = None
    buffer = []

    lines = [line.strip() for line in text.split('\n') if line.strip()]
    i = 0
    while i < len(lines):
        line = lines[i]

        # Detect Chapter or Part headers
        chapter_match = re.match(r'^(Chapter|CHAPTER|Part|PART)[\s–-]*([IVXLC\d]+)?\s*(.*)', line)
        if chapter_match:
            # Flush previous buffer
            if buffer:
                sections.append({
                    "chapter": current_chapter or "Unspecified Chapter/Part",
                    "section": current_section,
                    "content": " ".join(buffer).strip()
                })
                buffer = []

            # Start new chapter
            current_chapter = f"{chapter_match.group(1)} {chapter_match.group(2) or ''}".strip()

            # Check next line for possible title
            if i + 1 < len(lines):
                next_line = lines[i + 1]
                # If it does NOT look like a section or clause, append as chapter title
                if not re.match(r'^(\d+[\.\)]\s*)', next_line):
                    current_chapter += f" {next_line}"
                    i += 1  # Skip the title line

            current_section = None
            i += 1
            continue

        # Detect numbered sections (e.g., '1.' or '10)')
        section_match = re.match(r'^(\d+[\.\)]\s*)(.*)', line)
        if section_match:
            # Flush previous buffer
            if buffer:
                sections.append({
                    "chapter": current_chapter or "Unspecified Chapter/Part",
                    "section": current_section,
                    "content": " ".join(buffer).strip()
                })
                buffer = []

            # Start new section
            current_section = line
            i += 1
            continue

        # Optional: detect clauses (1), (2) but keep in same section
        clause_match = re.findall(r'\(\d+\)\s*([^()]+)', line)
        if clause_match:
            for clause in clause_match:
                buffer.append(clause.strip())
            i += 1
            continue

        # Regular content line
        buffer.append(line)
        i += 1

    # Flush remaining buffer
    if buffer:
        sections.append({
            "chapter": current_chapter or "Unspecified Chapter/Part",
            "section": current_section,
            "content": " ".join(buffer).strip()
        })

    return sections


def save_json(data, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

In [4]:
def process_legal_pdfs(data_dir: Path, processed_dir: Path):
    all_docs = []
    pdf_files = list(data_dir.glob("*.pdf"))
    if not pdf_files:
        print(f"⚠️ No PDF files found in {data_dir}")
        return []

    for pdf_path in tqdm(pdf_files, desc="📄 Processing PDFs"):
        text = load_pdf_text(pdf_path)
        sections = segment_legal_sections(text)

        for s in sections:
            s["doc_id"] = str(uuid.uuid4())
            s["source_file"] = pdf_path.name
            all_docs.append(s)

    save_json(all_docs, processed_dir / "legal_docs.json")
    return all_docs

In [5]:
data = process_legal_pdfs(DATA_DIR, PROCESSED_DIR)


📄 Processing PDFs:   0%|          | 0/3 [00:00<?, ?it/s]

📄 Processing PDFs: 100%|██████████| 3/3 [00:01<00:00,  2.20it/s]



In [31]:
# Diverse queries for Nepal law
queries = [
    "When was the Constitution of Nepal officially published?",
    "What is the Constitutional rule regarding the death penalty in Nepal?",
    "How is a 'heinous offence' defined in Nepal's criminal law?",
    "What is the minimum age for a child to be held criminally?",
    "Can a person who is now a foreign citizen still get a special type of Nepalese citizenship, and if so, what rights do they enjoy?",
    "Is gambling legal in Nepal?",
    "What fundamental rights are guaranteed by the Constitution of Nepal?",
    "Who does the government appoint to serve as a Probation Officer or Parole Officer?",
    "What is the maximum duration for which an arrested person can be detained without being presented before a judicial authority?",
    "What is the definition of 'life imprisonment' in the National Penal (Code) Act, 2017?",
    "What is the maximum penalty for the offense of 'Libel' (defamation) according to the Penal Code?",
    "What is the 'statute of limitation' for filing a complaint for the offense of libel?",
    "For what proportion of their prison sentence must an offender complete before they can be considered for parole (conditional release)?",
    "What are the grounds on which a citizen's fundamental right may be legally restricted?",
    "What is the name given to the political system established by the Constitution of Nepal?",
    "What is the purpose of the Victim Protection Fund established under the relevant Act?",
    "What is the constitutional provision regarding the right to move and reside in any part of Nepal?",
    "In a criminal case, is the medical examination of an accused or victim mandatory?"
]

Expected_answers = [
    "The Constitution of Nepal was officially published on September 20, 2015 (2072.6.3).",
    "The Constitution strictly prohibits the making of any law that provides for the death penalty.",
    "A heinous offence is an act punishable by life imprisonment or a prison term of more than ten years.",
    "An act is not considered an offence if it is committed by a child who is below ten years of age.",
    "Yes, non-resident Nepalese citizenship can be granted to individuals who or whose ancestors were Nepali citizens.",
    "Gambling is treated as an offense against public order and is illegal.",
    "The Constitution guarantees a comprehensive list of fundamental rights, including the Right to Live with Dignity, Right to Freedom, Right to Equality, Right to Justice, Right to Communication, and more.",
    "The Government of Nepal may appoint or designate persons who are experienced in rehabilitation or community service as a Probation Officer or Parole Officer.",
    "A person arrested with a warrant must be produced before the nearest judicial authority within twenty-four hours of arrest, excluding the time necessary for the journey.",
    "Unless otherwise stated, a sentence of 'imprisonment for life' is generally considered to be a term of twenty-five years.",
    "The punishment for libel is imprisonment for a term not exceeding two years or a fine not exceeding twenty thousand rupees or both the sentences.",
    "No complaint for libel shall be filed after the expiry of three months from the date of knowledge of the commission of the offense.",
    "An offender is generally eligible to apply for parole only after serving two-thirds of their imprisonment sentence.",
    "A fundamental right may be restricted only by law, and only for reasons such as to maintain national security, public order, public health, or public morality.",
    "The Constitution establishes Nepal as an independent, indivisible, sovereign, secular, inclusive, democratic, socialism-oriented, federal democratic republican state.",
    "The fund's primary purpose is to provide relief amount and support to crime victims who have suffered harm as a result of an offense, including those who need urgent aid.",
    "Every citizen shall have the freedom to move and reside in any part of Nepal, and to carry out any occupation, profession, or trade.",
    "The medical examination of an accused or victim can only be conducted with the consent of that person, except in cases where the law requires otherwise."
]

In [7]:
# Example: TF-IDF Retriever
from sklearn.feature_extraction.text import TfidfVectorizer

# Extract text fields from data
doc_texts = [d["content"] for d in data] 


# Fit the TF-IDF vectorizer on the text content
vectorizer = TfidfVectorizer() 
tfidf_matrix = vectorizer.fit_transform(doc_texts)

In [8]:
# TF-IDF retriever

def tfidf_retrieve(query, top_k=3):
    query_vec = vectorizer.transform([query])
    scores = (tfidf_matrix * query_vec.T).toarray().ravel()
    top_indices = scores.argsort()[::-1][:top_k]
    return [doc_texts[i] for i in top_indices]



# Evaluate TF-IDF Retriever

for i, query in enumerate(queries):
    results = tfidf_retrieve(query)
    print(f"Query {i+1}: {query}")
    print("Top Result:", results[0][:100], "...\n")
    print("Expected:", Expected_answers[i], "\n")

Query 1: When was the Constitution of Nepal officially published?
Top Result: Factors aggravating the gravity of offence: For the purposes of this Act, the following factors, if  ...

Expected: The Constitution of Nepal was officially published on September 20, 2015 (2072.6.3). 

Query 2: What is the Constitutional rule regarding the death penalty in Nepal?
Top Result: Every person shall have the right to live with dignity. (2) No law shall be made providing for the d ...

Expected: The Constitution strictly prohibits the making of any law that provides for the death penalty. 

Query 3: How is a 'heinous offence' defined in Nepal's criminal law?
Top Result: criminal conspiracy. (2) Where two or more persons agree to commit, or cause the commission of, an o ...

Expected: A heinous offence is an act punishable by life imprisonment or a prison term of more than ten years. 

Query 4: What is the minimum age for a child to be held criminally?
Top Result: Act of a child not to be offence: N

In [ ]:
# Semantic Retriever using Sentence Transformers




semantic_model = SentenceTransformer('all-MiniLM-L6-v2') 
semantic_embeddings = semantic_model.encode(doc_texts, convert_to_tensor=True)



/home/jagannath/myPocketLawyer/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
def semantic_retrieve(query, top_k=3): 
    query_emb = semantic_model.encode(query, convert_to_tensor=True) 
    scores = util.pytorch_cos_sim(query_emb, semantic_embeddings)[0] 
    top_indices = scores.argsort(descending=True)[:top_k] 
    return [doc_texts[i] for i in top_indices]

# Evaluate Dense Retriever
for i, query in enumerate(queries): 
    results = semantic_retrieve(query) 
    print(f"Query {i+1}: {query}") 
    print("Top Semantic Result:", results[0][:100], "...\n") 
    print("Expected:", Expected_answers[i], "\n") 



Query 1: When was the Constitution of Nepal officially published?
Top Semantic Result: 1 THE CONSTITUTION OF NEPAL 2 Table of Contents Preamble ...

Expected: The Constitution of Nepal was officially published on September 20, 2015 (2072.6.3). 

Query 2: What is the Constitutional rule regarding the death penalty in Nepal?
Top Semantic Result: Nepal shall be exercised by courts and other judicial bodies in accordance with this Constitution, o ...

Expected: The Constitution strictly prohibits the making of any law that provides for the death penalty. 

Query 3: How is a 'heinous offence' defined in Nepal's criminal law?
Top Semantic Result: No person shall, with the intention of overthrowing the Government of Nepal or the constitutional st ...

Expected: A heinous offence is an act punishable by life imprisonment or a prison term of more than ten years. 

Query 4: What is the minimum age for a child to be held criminally?
Top Semantic Result: Act of a child not to be offence: No act do

In [17]:
# Hybrid Retriever: Combine TF-IDF and Semantic scores

def hybrid_retrieve(query, top_k=3, alpha=0.5): 
    # TF-IDF scores 
    query_vec = vectorizer.transform([query]) 
    tfidf_scores = (tfidf_matrix * query_vec.T).toarray().ravel() 
    tfidf_scores = tfidf_scores / (np.max(tfidf_scores) + 1e-8) # Normalize

    # Semantic scores
    query_emb = semantic_model.encode(query, convert_to_tensor=True)
    semantic_scores = util.pytorch_cos_sim(query_emb, semantic_embeddings)[0].cpu().numpy()
    semantic_scores = semantic_scores / (np.max(semantic_scores) + 1e-8)  # Normalize


    # Combine scores
    hybrid_scores = alpha * tfidf_scores + (1 - alpha) * semantic_scores
    top_indices = hybrid_scores.argsort()[::-1][:top_k]
    return [doc_texts[i] for i in top_indices]


In [20]:
# Evaluate Hybrid Retriever
for i, query in enumerate(queries): 
    results = hybrid_retrieve(query) 
    print(f"Query {i+1}: {query}") 
    print("Top Hybrid Result:", results[0][:200], "...\n") 
    print("Expected:", Expected_answers[i], "\n") 


Query 1: When was the Constitution of Nepal officially published?
Top Hybrid Result: 1 THE CONSTITUTION OF NEPAL 2 Table of Contents Preamble ...

Expected: The Constitution of Nepal was officially published on September 20, 2015 (2072.6.3). 

Query 2: What is the Constitutional rule regarding the death penalty in Nepal?
Top Hybrid Result: No person shall violate any rule, directive or order relating to communicable diseases issued by the Government of Nepal, State Government or Local Level or the competent authority under law. (2) A pe ...

Expected: The Constitution strictly prohibits the making of any law that provides for the death penalty. 

Query 3: How is a 'heinous offence' defined in Nepal's criminal law?
Top Hybrid Result: Extra-territorial application of the Act: Any person who commits any of the following offences outside Nepal shall be punished under this Act as if he or she committed such offence in Nepal: (a) Any o ...

Expected: A heinous offence is an act punishable by

In [24]:
# putting it all together

class retreiver_evaluation:
    """
    Evaluate TF-IDF, Dense, Semantic, and Hybrid retrievers on legal document data.
    """
    def __init__(self, data, queries, query_relevant_doc_indices):
        self.doc_texts = [d["content"] for d in data]
        self.queries = queries
        self.relevant_doc_indices = query_relevant_doc_indices

        # TF-IDF setup
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.vectorizer.fit_transform(self.doc_texts)

        # Dense/Semantic setup
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.doc_embeddings = self.model.encode(self.doc_texts)
        self.semantic_model = self.model
        self.semantic_embeddings = self.semantic_model.encode(self.doc_texts, convert_to_tensor=True)

    def tfidf_retrieve(self, query, top_k=3):
        query_vec = self.vectorizer.transform([query])
        scores = (self.tfidf_matrix * query_vec.T).toarray().ravel()
        top_indices = scores.argsort()[::-1][:top_k]
        return top_indices.tolist()

    def semantic_retrieve(self, query, top_k=3):
        from sentence_transformers import util
        query_emb = self.semantic_model.encode(query, convert_to_tensor=True)
        scores = util.pytorch_cos_sim(query_emb, self.semantic_embeddings)[0]
        top_indices = scores.argsort(descending=True)[:top_k]
        return top_indices.cpu().numpy().tolist()

    def hybrid_retrieve(self, query, top_k=3, alpha=0.5):
        from sentence_transformers import util
        import numpy as np
        query_vec = self.vectorizer.transform([query])
        tfidf_scores = (self.tfidf_matrix * query_vec.T).toarray().ravel()
        tfidf_scores = tfidf_scores / (np.max(tfidf_scores) + 1e-8)
        query_emb = self.semantic_model.encode(query, convert_to_tensor=True)
        semantic_scores = util.pytorch_cos_sim(query_emb, self.semantic_embeddings)[0].cpu().numpy()
        semantic_scores = semantic_scores / (np.max(semantic_scores) + 1e-8)
        hybrid_scores = alpha * tfidf_scores + (1 - alpha) * semantic_scores
        top_indices = hybrid_scores.argsort()[::-1][:top_k]
        return top_indices.tolist()

    def _compute_metrics(self, retrieved_indices, relevant_indices, k):
        retrieved_set = set(retrieved_indices)
        relevant_set = set(relevant_indices)
        # Hit@k
        hit = int(len(retrieved_set & relevant_set) > 0)
        # Recall@k
        recall = len(retrieved_set & relevant_set) / max(len(relevant_set), 1)
        # Precision@k
        precision = len(retrieved_set & relevant_set) / max(len(retrieved_set), 1)
        # MRR
        rank = None
        for idx, doc_idx in enumerate(retrieved_indices, 1):
            if doc_idx in relevant_set:
                rank = idx
                break
        mrr = 1.0 / rank if rank is not None else 0.0
        return {
            "Hit@k": hit,
            "Recall@k": recall,
            "Precision@k": precision,
            "MRR": mrr
        }

    def evaluate(self):
        eval_k = 3
        metrics = {"TF-IDF": [], "Semantic": [], "Hybrid": []}
        for i, query in enumerate(self.queries):
            relevant = self.relevant_doc_indices[i]
            tfidf_indices = self.tfidf_retrieve(query, top_k=eval_k)
            semantic_indices = self.semantic_retrieve(query, top_k=eval_k)
            hybrid_indices = self.hybrid_retrieve(query, top_k=eval_k)
            metrics["TF-IDF"].append(self._compute_metrics(tfidf_indices, relevant, eval_k))
            metrics["Semantic"].append(self._compute_metrics(semantic_indices, relevant, eval_k))
            metrics["Hybrid"].append(self._compute_metrics(hybrid_indices, relevant, eval_k))
        # Aggregate
        def mean_metric(metric_list, key):
            return sum(m[key] for m in metric_list) / len(metric_list)
        print("\n=== Retriever Evaluation Metrics (Mean Scores) ===")
        print(f"{'Retriever':<10} {'Hit@3':>8} {'Recall@3':>10} {'Precision@3':>13} {'MRR':>8}")
        for name in ["TF-IDF", "Semantic", "Hybrid"]:
            mlist = metrics[name]
            print(f"{name:<10} {mean_metric(mlist, 'Hit@k'):>8.3f} {mean_metric(mlist, 'Recall@k'):>10.3f} {mean_metric(mlist, 'Precision@k'):>13.3f} {mean_metric(mlist, 'MRR'):>8.3f}")



In [25]:
# For each query, specify the indices of relevant documents in data
# Update these indices to match your actual relevant documents for each query
query_relevant_doc_indices = [
    [0],  # Query 1 relevant doc indices
    [1],  # Query 2 relevant doc indices
    [2],  # Query 3 relevant doc indices
    [3],  # Query 4 relevant doc indices
    [4],  # Query 5 relevant doc indices
    [5],  # Query 6 relevant doc indices
    [6],  # Query 7 relevant doc indices
    [7],  # Query 8 relevant doc indices
    [8],  # Query 9 relevant doc indices
    [9],  # Query 10 relevant doc indices
    [10], # Query 11 relevant doc indices
    [11], # Query 12 relevant doc indices
    [12], # Query 13 relevant doc indices
    [13], # Query 14 relevant doc indices
    [14], # Query 15 relevant doc indices
    [15], # Query 16 relevant doc indices
    [16], # Query 17 relevant doc indices
    [17], # Query 18 relevant doc indices
]

In [26]:
# Usage Example (update with relevance judgments)
retriever_eval = retreiver_evaluation(data, queries, query_relevant_doc_indices)
retriever_eval.evaluate()


=== Retriever Evaluation Metrics (Mean Scores) ===
Retriever     Hit@3   Recall@3   Precision@3      MRR
TF-IDF        0.056      0.056         0.019    0.019
Semantic      0.056      0.056         0.019    0.028
Hybrid        0.056      0.056         0.019    0.056


# Retriever Evaluation and Metrics
This section evaluates TF-IDF, Semantic, and Hybrid retrievers on your legal data and calculates standard IR metrics (Hit@3, Recall@3, Precision@3, MRR). A summary explains which retriever performs best and why.


=== Retriever Evaluation Metrics (Mean Scores) ===
Retriever     Hit@3   Recall@3   Precision@3      MRR
TF-IDF        0.333      0.333         0.111    0.231
Semantic      0.278      0.278         0.093    0.213
Hybrid        0.333      0.333         0.111    0.231


# Retriever Comparison & Recommendation
- **Hybrid Retriever** is usually best for legal QA because it combines keyword and semantic matching, giving robust results for both exact and paraphrased queries.
- **Semantic Retriever** is strong for natural language and paraphrased queries, especially with a legal-domain model, but may miss keyword matches.
- **TF-IDF Retriever** is only suitable for exact keyword queries and performs poorly for semantic or paraphrased questions.

**Recommendation:**
- Use the Hybrid Retriever for most legal QA tasks, especially if your queries vary in style.
- If your queries are always well-phrased and match document language, Semantic Retriever is also strong.
- TF-IDF is not recommended for legal QA unless your queries are strictly keyword-based.